In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys
import os
import math
import spacy
import pandas as pd
import tensorflow as tf
sys.path.append("../../python/")

In [2]:
from workbench.data.dataframe import TextDataFrame
from workbench.tf.data.text_dataset import *
from workbench.data.nlp.spacy import *

In [3]:
original_train_data = pd.read_csv("input/train.csv")
original_test_data = pd.read_csv("input/test.csv")

In [4]:
if not os.path.exists("input/train_tokenized.csv"):
    nlp_processed_train_data = tokenize(original_train_data, "text")
    nlp_processed_train_data.to_csv("input/train_tokenized.csv")

In [5]:
if not os.path.exists("input/test_tokenized.csv"):
    nlp_processed_test_data =  tokenize(original_test_data, "text")
    nlp_processed_test_data.to_csv("input/test_tokenized.csv")

In [6]:
# nlp_processed_test_data

In [7]:
text_col = "nlp_processed"
dataset_df  = TextDataFrame(train_df_path="input/train_tokenized.csv", 
                 test_df_path="input/test_tokenized.csv", 
                 text_col=text_col, 
                 category_col='author')

Fitting LabelEncoder and LabelBinarizer...
Done!
Splitting the data set(stratified sampling)...
Done!


In [8]:
full_df = dataset_df._get_full_train_df()
full_df["length"] = full_df["text"].apply(lambda x : len(x.split()))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [9]:
full_df.count()

Unnamed: 0       19579
id               19579
text             19579
author           19579
nlp_processed    19579
length           19579
dtype: int64

In [10]:
# Show debugging output
tf.logging.set_verbosity(tf.logging.DEBUG)

# Set default flags for the output directories
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
    flag_name='model_dir', 
    default_value='./spooky_tf_models',
    docstring='Output directory for model and training stats.')

tf.app.flags.DEFINE_string(
    flag_name='data_dir', 
    default_value='./spooky_data',
    docstring='Directory to download the data to.')

BATCH_SIZE = 64


In [11]:
MAX_DOCUMENT_LENGTH = 150
#What would be the leftout documents if we choose 'MAX_DOCUMENT_LENGTH' as doc length
full_df[full_df["length"] > MAX_DOCUMENT_LENGTH].count()[0]

16

In [12]:
%time
VOCAB_SIZE = save_vocab(dataset_df.train_df[text_col].as_matrix(), 
                       outfilename='horror_vocab.tsv', 
                       MAX_DOCUMENT_LENGTH=MAX_DOCUMENT_LENGTH) #

VOCAB_SIZE

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


Widget Javascript not detected.  It may not be installed or enabled properly.



25140 words into horror_vocab.tsv


25140

In [13]:
train_input_fn, train_input_hook = setup_input_graph(dataset_df._get_train_data(), 
                                                     dataset_df._get_one_hot_train_label(),
                                                    batch_size=BATCH_SIZE, 
                                                     scope='train-data')

Size of train data: 2.389MB
Labels and their document counts based on author
EAP    6320
HPL    4508
MWS    4835
Name: author, dtype: int64


In [14]:
eval_input_fn, eval_input_hook =  setup_input_graph(dataset_df._get_val_data(), 
                                                    dataset_df._get_one_hot_val_label(),
                                                    batch_size=BATCH_SIZE, 
                                                    scope='eval-data')

Size of validation data: 0.601MB
Labels and their document counts based on author
EAP    1580
HPL    1127
MWS    1209
Name: author, dtype: int64


In [15]:
from workbench.tf.text_classification.multi_class_cnn_rnn import TextCNNRNN
model = TextCNNRNN("horror_vocab.tsv", 
                 VOCAB_SIZE, 
                 train_input_fn, 
                 train_input_hook, 
                 eval_input_fn, 
                 eval_input_hook,
                   train_steps=4500,
                   min_eval_frequency=100,
                  max_doc_length=MAX_DOCUMENT_LENGTH)

In [16]:
# from workbench.tf.text_classification.bilstm import BiLstm
# model = BiLstm("horror_vocab.tsv", 
#                  VOCAB_SIZE, 
#                  train_input_fn, 
#                  train_input_hook, 
#                  eval_input_fn, 
#                  eval_input_hook,
#                    train_steps=7000,
#                    min_eval_frequency=300,
#                   max_doc_length=MAX_DOCUMENT_LENGTH)

In [17]:
# from workbench.tf.text_classification.fast_text import FastText
# model = FastText("horror_vocab.tsv", 
#                  VOCAB_SIZE, 
#                  train_input_fn, 
#                  train_input_hook, 
#                  eval_input_fn, 
#                  eval_input_hook,
#                    train_steps=2000,
#                    min_eval_frequency=50,
#                   max_doc_length=MAX_DOCUMENT_LENGTH)

In [18]:
%time
# try:
model.run()
# except:
#     print('Trinaing done!')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5287398940>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 128, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './spooky_tf_models'}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
DEBUG:tensorflow:features -----> Tensor("train-data/IteratorGetNext:0", shape=(?,), dtype=string, device=/device:CPU:0)
INFO:tensorflow:table info: <tensorflow.python.ops.lookup_ops.IdTableWithHashBuckets 

INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-11-07-12:39:06
INFO:tensorflow:Saving dict for global s

INFO:tensorflow:wide_layer: ------> Tensor("hidden-mlp-layer/dropout/Identity:0", shape=(?, 1024), dtype=float32)
INFO:tensorflow:conv1: ------> Tensor("conv_layer/conv2d/Relu:0", shape=(?, 32, 32, 32), dtype=float32)
INFO:tensorflow:pool1: ------> Tensor("conv_layer/max_pooling2d/MaxPool:0", shape=(?, 16, 16, 32), dtype=float32)
INFO:tensorflow:conv2: ------> Tensor("conv_layer/conv2d_2/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
INFO:tensorflow:pool2: ------> Tensor("conv_layer/max_pooling2d_2/MaxPool:0", shape=(?, 8, 8, 64), dtype=float32)
INFO:tensorflow:pool2_flat: ------> Tensor("conv_layer/Reshape_1:0", shape=(?, 4096), dtype=float32)
INFO:tensorflow:logits: ------> Tensor("logits-layer/dense/BiasAdd:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:predicted_class: ------> Tensor("output-layer/class_output:0", shape=(?,), dtype=int64)
INFO:tensorflow:predicted_probabilities: ------> Tensor("output-layer/softmax_output:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Summary n

INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-11-07-12:41:45
INFO:tensorflow:Saving dict for global step 401: Accuracy = 0.4075, Precision = 0.0, Recall = 0.0, global_step = 401, loss = 0.869199
INFO:tensorflo

...

INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-11-07-12:46:41
INFO:tensorflow:Saving dict for global step 1201: Accuracy = 0.404531, Precision = 0.0, Recall = 0.0, global_step = 1201, loss = 0.86957
INFO:tensorflow:Validation (step 1300): Accuracy = 0.404531, Precision = 0.0, Recall = 0.0, loss = 0.86957, global_step = 1201
INFO:tensorflow:Saving checkpoints for 1301 into ./spooky_tf_models/model.ckpt.
INFO:tensorflow:glob

INFO:tensorflow:conv2: ------> Tensor("conv_layer/conv2d_2/Relu:0", shape=(?, 16, 16, 64), dtype=float32)
INFO:tensorflow:pool2: ------> Tensor("conv_layer/max_pooling2d_2/MaxPool:0", shape=(?, 8, 8, 64), dtype=float32)
INFO:tensorflow:pool2_flat: ------> Tensor("conv_layer/Reshape_1:0", shape=(?, 4096), dtype=float32)
INFO:tensorflow:logits: ------> Tensor("logits-layer/dense/BiasAdd:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:predicted_class: ------> Tensor("output-layer/class_output:0", shape=(?,), dtype=int64)
INFO:tensorflow:predicted_probabilities: ------> Tensor("output-layer/softmax_output:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Summary name lstm-layer/rnn/while/Exit_2:0 is illegal; using lstm-layer/rnn/while/Exit_2_0 instead.
INFO:tensorflow:Summary name output-layer/softmax_output:0 is illegal; using output-layer/softmax_output_0 instead.
INFO:tensorflow:Starting evaluation at 2017-11-07-12:47:42
INFO:tensorflow:Restoring parameters from ./spooky_tf_models/model

INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [55/100]
INFO:tensorflow:Evaluation [56/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-11-07-12:49:16
INFO:tensorflow:Saving dict for global step 1601: Accuracy = 0.413594, Precision = 0.0, Recall = 0.0, global_step = 1601, loss = 0.867782
INFO:tensorflow:Validation (step 1700): Accuracy = 0.413594, Precision = 0.0, Recall = 0.0, loss = 0.867782, global_step = 1601
INFO:tensorflow:Saving checkpoints for 1701 into ./spooky_tf_models/model.ckpt.
INFO:tensorflow:global_step/sec: 2.50254
INFO:tensorflow:loss = 0.846633, step = 1701 (39.959 sec)


KeyboardInterrupt: 

# Test Data

In [ ]:
test_input_fn =  test_inputs(dataset_df._get_test_data(), 
                                        batch_size=1, 
                                        scope='test-data')

In [ ]:
predictions_fn = model.predict(test_input_fn)

In [ ]:
%time
predictions = []
classes = []

for r in predictions_fn:
    predictions.append(r['probabilities'])
    classes.append(r['classes'])

In [ ]:
ids = dataset_df.test_df['id']
results = pd.DataFrame(predictions, columns=['EAP', 'HPL','MWS'])
results.insert(0, "id", ids)


In [ ]:
results.head()


In [ ]:
results.to_csv("fast_text_tokenized.csv", index=False)

# Train Data Prediction for ensembling

In [ ]:
train_input_fn_2 =  test_inputs(dataset_df._get_full_train_df()['text'].as_matrix(), 
                                        batch_size=1, 
                                        scope='test-data')

In [ ]:
predictions_fn2 = model.predict(train_input_fn_2)

In [ ]:
%time
predictions = []
classes = []

for r in predictions_fn2:
    predictions.append(r['probabilities'])
    classes.append(r['classes'])

In [ ]:
ids =dataset_df._get_full_train_df()['id']
results = pd.DataFrame(predictions, columns=['EAP', 'HPL','MWS'])
results.insert(0, "id", ids)
results.head()

In [ ]:
results.to_csv("trainset_fast_text_text_pos_bigrams.csv", index=False)

In [ ]:
results.head()